In [258]:
import PIL.Image as IMG
import PIL.ImageFont as ImageFont
import PIL.ImageDraw as ImageDraw
from io import BytesIO
from rembg import remove
import cv2 as cv
import os
import warnings
warnings.filterwarnings("ignore")

# Image Transformations

In [259]:
def Resize(image, new_width=200):
    """
    Resizes the image
    """
    width, height = image.size
    aspect_ratio = height / width
    new_height = int(new_width * aspect_ratio)
    resized_image = image.resize((new_width, new_height))
    return resized_image

def EraseBKG(image):
    """
    Removes the background from the image
    Requires rembg python library
    """
    # Convert Pillow Image to bytes
    image_bytes = BytesIO()
    image.save(image_bytes, format="PNG")
    image_bytes = image_bytes.getvalue()
    
    # Remove the background
    result_bytes = remove(image_bytes)
    
    # Convert the result (bytes) back to a Pillow Image object
    result_image = IMG.open(BytesIO(result_bytes))
    return result_image

def GrayScale(image):
    """
    Returns a grayscale image. Each pixel is a result of the mean of its RGB values
    """
    grayscale_image = image.convert("L")
    return grayscale_image

# Video Transformations

In [260]:
def getFrames(sec, video_path, count):
    """
    Splits and exports frames from the input video    
    """
    vidcap = cv.VideoCapture(video_path)
    vidcap.set(cv.CAP_PROP_POS_MSEC, sec*1000)
    hasFrames, image = vidcap.read()
    
    if hasFrames:
        cv.imwrite("_inputs/frames/" + str(count) + ".jpg", image)
        return hasFrames

def ConvertVideoToFrames(video_path):
    """
    Setup function to clear and convert to frames
    @Prerequisite - getFrames()
    """
    ClearDir("_inputs/frames")
    
    sec = 0
    vidSeq_div = 0.5
    count = 1
    success = getFrames(sec, video_path, count)
    while success:
        count += 1
        sec += vidSeq_div
        sec = round(sec, 2)
        success = getFrames(sec, video_path, count)

def ConvertASCIIFramesToVideo():
    pass

# ASCIIfy

In [261]:
def ASCIIify(image, new_width):
    """
    Returns ASCII version of the image.
    
    To change the characters of the ASCII image, change the 'symbols' parameter.
    """
    symbols = list(" $@B%8&WM*#oahkbdpqwmZO0QLCJUYXzcvunxrjft/\\|(){}[]-?_+~i!lI;:,^`.")
    
    # Define the scaling factor
    scaling_factor = 256 // len(symbols)
    
    # Get pixel values
    pixels = image.getdata()
    
    # Map pixel values to symbols
    characters = "".join([symbols[min(pixel // scaling_factor, len(symbols) - 1)] for pixel in pixels])
    pixel_count = len(characters)
    
    # Convert to lines of ASCII
    ASCII_image = "\n".join(characters[i:(i + new_width)] for i in range(0, pixel_count, new_width))
    return ASCII_image


def Export_txt(image):
    """
    Converts the output from ASCIIify() to a .txt file
    """
    resized_image = Resize(image)
    new_width = 200  # Set the desired width for the ASCII art
    ASCII_art = ASCIIify(GrayScale(EraseBKG(resized_image)), new_width)

    # Saving as text file
    base_name = "_outputs/TXT_"
    extension = ".txt"
    
    counter = 1
    output_file = f"{base_name}{counter}{extension}"
    
    while os.path.exists(output_file):
        counter += 1
        output_file = f"{base_name}{counter}{extension}"

    with open(output_file, "w") as f:
        f.write(ASCII_art)
    
    return ASCII_art

def Export_img(ascii_art):
    """
    Converts the output from Export_txt() to a .png file
    """
    font_size = 10  # Set the font size for the characters in the image
    font = ImageFont.truetype("cour.ttf", font_size)  # Load a font (e.g., Courier)
    
    # Calculate image dimensions
    lines = ascii_art.split("\n")
    line_height = font.getbbox("A")[3] - font.getbbox("A")[1]  # Calculate the height of a single line of text
    width = max(font.getbbox(line)[2] - font.getbbox(line)[0] for line in lines)  # Calculate the max line width
    height = len(lines) * line_height  # Calculate the total height of the image
    
    # Create a new image with a white background
    new_image = IMG.new("RGB", (width, height), (255, 255, 255))
    draw = ImageDraw.Draw(new_image)
    
    # Draw each line of text
    y = 0
    for line in lines:
        draw.text((0, y), line, fill=(0, 0, 0), font=font)
        y += line_height
    
    # Save the image
    base_name = "_outputs/IMG_"
    extension = ".jpg"
    
    # Start numbering from 1
    counter = 1
    output_file = f"{base_name}{counter}{extension}"
    
    # Check for existing files and find the next available number
    while os.path.exists(output_file):
        counter += 1
        output_file = f"{base_name}{counter}{extension}"
    
    # Save the image with the unique sequential filename
    new_image.save(output_file)

# Helper functions

In [262]:
def ImageToASCII(image):
    
    # Generate ASCII text
     ascii_text = Export_txt(image)
     
     # Generate ASCII PNG
     Export_img(ascii_text)
        

In [265]:
def ClearDir(directory):
    """
    Clears the contents in the target directory
    """
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        os.remove(filepath)
                
def Detect_Input(file_path):
    """
    Distinguishes between the input given, i.e, photo or a video, else None
    """
    # Get the file extension
    file_extension = file_path.split('.')[-1].lower()
    image_extensions = ['jpg', 'jpeg', 'png', 'gif', 'bmp']
    video_extensions = ['mp4', 'avi', 'mov', 'mkv', 'wmv']

    ClearDir("_outputs")
    
    # Input is a IMAGE
    if file_extension in image_extensions:        
        image = IMG.open(file_path)
        ImageToASCII(image)

        print(f"Check '/_outputs'")
        
    # Input is a VIDEO
    elif file_extension in video_extensions:
        ClearDir("_inputs/frames")
        
        # Convert video to frames
        ConvertVideoToFrames(file_path)
        print("Frames ready")

        # Converting frames to ASCII
        for filename in os.listdir("_inputs/frames/"):
            filepath = os.path.join("_inputs/frames/", filename)
            image = IMG.open(filepath)
            ImageToASCII(image)

        # Convering ASCII frames to back to Video:
        ConvertASCIIFramesToVideo()

        print(f"Check '/_outputs'")
        
    # INPUT not IMAGE or VIDEO
    else:
        print(f"Enter a valid media format from below\n{image_extensions} or {video_extensions}")
        return None

def __main__(filepath):
    Detect_Input(filepath)

In [266]:
__main__("_inputs/userInput/video.mp4")

Frames ready
Check '/_outputs'
